## Содержание: <a class="anchor" id="contents"></a>
* [Обработка корпуса](#corpus)
* [Визуализация предложения из корпуса](#visual)
* [Генерация и обработка грамматики](#grammar)
* [Генерация рекурсивной сети переходов](#rtn)
* [Генерация JSON](#json)

### Обработка корпуса: <a class="anchor" id="corpus"></a>

In [237]:
from xml.dom.minidom import (parse, Element)

In [238]:
#corpus_filename = 'Glavnaya_taina_nachala_voiny'
#corpus_filename = 'pedagogika'

corpus_filename = 'sirija'

file_ext = '.xml'
#file_ext = '.tgt'
doc = parse(corpus_filename + file_ext)
sentences = doc.getElementsByTagName('S') # все элементы с тегом S (все предложения корпуса)

orig_name = corpus_filename

In [239]:
import os
folder = corpus_filename + '_ready/'
if not os.path.exists(folder):
    os.makedirs(folder)
corpus_filename =  folder + corpus_filename

In [240]:
f'Количество предложений в корпусе = {len(sentences)}'

'Количество предложений в корпусе = 79'

In [241]:
import time
t = time.time()

In [242]:
d = {} # словарь всей грамматики граммема - слова
non_projective = set()
pj = 0

class Sentence:
    def __init__(self, sent: Element):
        self.id = sent.getAttribute('ID')
        self.words = []
        
        self.parseWords(sent.getElementsByTagName('W'))
        # получаем все элементы xml с тегом W (слова)
        
    def parseWords(self, rawWords):
        
        self.wordMap = {} # для последующего связывания слов
        # ключ - доминатор(айди доминатора) этого слова
        # значение - список слов, для которых доминатор равен ключу
        
        for rawWord in rawWords:
            
            word = Word(rawWord, self.id) # переносим в класс
            
            self.words.append(word) # добавляем в общий список слов
            
            if word.dom == '_root':
                self.rootWord = word # корневое слово
                
            # добавление в мапу
            if word.dom in self.wordMap.keys():
                self.wordMap[word.dom].append(word)
            else:
                self.wordMap[word.dom] = [word]

        # цикл по всем словам, чтобы связать их с помощью мапы
        for word in self.words:
            if word.id in self.wordMap.keys():
                for d in self.wordMap[word.id]:
                    word.addWord(d)
    def printGram(self):
        # печать грамматики предложения = печать грамматики одного слова(которое печатает все свои связанные слова, то есть все слова в предложении)
        self.rootWord.printGram(True) 
        
        
class Word:
    def __init__(self, w: Element, sentId):
        self.dom = w.getAttribute('DOM')
        self.feat = w.getAttribute('FEAT')
        self.feat = self.feat.replace(' МЕТА', '').replace(' НАСТ ', ' ').replace(' СЛ', '').replace('COM', '')
        
        if self.feat == '':
            self.feat = 'S'
        else:
            self.feat = self.feat.split(' ')[0]
            
        self.id = w.getAttribute('ID')
        self.lemma = w.getAttribute('LEMMA')
        self.link = w.getAttribute('LINK')
        
        v = " ".join(t.nodeValue for t in w.childNodes if t.nodeType == t.TEXT_NODE).lower().strip()
        # v - просто слово
        
        if len(v) != 0 and v[-1] == '.':
            v = v[:-1]
        if v != '':
            # добавление слова в словарь
            if self.feat in d.keys():
                d[self.feat].add(v)
            else:
                d[self.feat] = {v}
        self.v = v
        
        
        self.sentId = sentId
        
        self.connectedWords = [] #связанные с этим словом слова
        
    def addWord(self, w):
        # добавление слова в связанные с этим словом слова
        self.connectedWords.append(w)
        
    def printGram(self, checkProj):
        if checkProj:
            #t = checkProjectivity(self, {}, 0)
            global seen_ids
            seen_ids = set()
            t = is_non_projective(self)
            if t:
                #global pj
                #pj += 1
                
                #print('not proj')
                # закомментировать return, чтобы такие предложения не пропускались
                #print(f'\nNOT PROJECTIVE, {self.sentId}\n')
                non_projective.add(self.sentId)
                return
            print('R', end='')
        print('D{' + f'[{self.feat}' + ']}->', end = '') # F(t) ->
        
        if self.connectedWords == []:
            print(f'[{self.feat}]\n', end = '') # когда узел - терминальный
            
        else:
            t = False # для расположения граммем слова в нужном месте
            for w in self.connectedWords:
                
                # проход по всем словам в связанных словах
                if not t and (int(w.id) > int(self.id)): 
                    # когда этого не происходило ранее и когда айди текущего связанного слова превысило айди самого слова
                    print(f'[{self.feat}]', end = '')
                    t = True
                     
                print(';D{[' + f'{w.feat}' + '], ' + f'{w.link}' + '};', end = '') # D(t, s)
            if not t:
                print(f'[{self.feat}]', end = '')
            for w in self.connectedWords:
                s = 'D'
                #if w.dom == '_root':
                #    s = 'RF'
                print('\nD{[' + f'{w.feat}], {w.link}' + '}->' + s + '{[' + f'{w.feat}' + ']}') 
                # D(t, s) -> F(t)
                #print() # раскомментировать и закомментировать предыдущую, если не хотим печатать D(t, s) -> F(t)
                w.printGram(False)

seen_ids = set()

def is_non_projective(w):
    for v in [f for f in w.connectedWords if int(f.id) < int(w.id)]:
        t = is_non_projective(v)
        if t:
            return True
        
    if any(map(lambda x: int(x) > int(w.id), seen_ids)):
        return True
    seen_ids.add(w.id)
    
    for v in [f for f in w.connectedWords if int(f.id) > int(w.id)]:
        t = is_non_projective(v)
        if t:
            return True
    
    return False
    
    

### Визуализация дерева: <a class="anchor" id="visual"></a>

In [243]:
draw_tree = False

In [244]:
if draw_tree:
    import graphviz

    def drawSentenceTree(sentence_id, open_pdf = False):
        sentence = Sentence(sentences[sentence_id - 1])
        SCALE_FACTOR = 2
        grviz = graphviz.Digraph(f'dependency_tree_{sentence.id}', engine="neato",)
        def addWordsToTree(grviz, word, level):
            x_position = int(word.id) * SCALE_FACTOR
            y_position = -level * SCALE_FACTOR
            if word.dom != '_root':
                grviz.edge(word.dom, word.id, label=word.link)
            grviz.node(word.id, word.id + '\n' + word.v + '\n' + word.feat, pos=f"{x_position},{y_position}!")
            for child in word.connectedWords:
                addWordsToTree(grviz, child, level + 1)
        
        
        
        addWordsToTree(grviz, sentence.rootWord, 0)
        if open_pdf:
            grviz.view()
        return grviz
    

In [245]:
if draw_tree:
    grviz = drawSentenceTree(6, open_pdf = True)

In [246]:
if draw_tree:
    grviz

### Генерация и обработка грамматики: <a class="anchor" id="grammar"></a>

In [247]:
def parseSentences(sentence_id = None):
    if sentence_id != None:
        parseSentence(sentences[sentence_id - 1])
    else:
        for sent in sentences:
            # пробегаемся по всем предложениям корпуса
            parseSentence(sent)

def parseSentence(sent: Element):
    sentence = Sentence(sent)
    sentence.printGram()

In [248]:
import sys
#corpus_filename = corpus_filename + '_48'
filename = corpus_filename + '_' + 'gram.out'
#filename = 'test_test_test.txt'

In [249]:

orig_stdout = sys.stdout
f = open(filename, 'w', encoding = 'utf-8')

sys.stdout = f
# перенаправление stdout в файл, чтобы все печаталось в файл
#pj = 0
#parseSentences(sentence_id=6)
parseSentences()

#print(pj)

# перенаправление обратно
sys.stdout = orig_stdout
f.close()

In [250]:
t_after_gram = time.time()
print(f'Время, потраченное на генерацию грамматики - {t_after_gram - t}')

Время, потраченное на генерацию грамматики - 0.08650016784667969


In [251]:
f'Количество непроективных деревьев в {corpus_filename} {len(non_projective)}'

'Количество непроективных деревьев в sirija_ready/sirija 5'

In [252]:
filename_sorted = corpus_filename + '_' + 'gram_sorted.out'

In [253]:
def process_rules_from_file(file_path):
    with open(file_path, 'r', encoding='utf8') as file:
        rules = [line.strip() for line in file if line.strip()]

    return process_rules(rules)

def process_rules(rules):
    rules = rules[::-1]

    replacements = {}
    to_del = []

    for i in range(len(rules)):

        left, right = rules[i].split('->')
        left = left.strip()
        right = right.strip()
        
        if right in replacements:
            if 'D{' not in replacements[right]:
                rules[i] = f"{left}->{replacements[right]}"
                to_del.append(f"{right}->{replacements[right]}")
        else:
            replacements[left] = right
    i = 0
    while i < len(rules):
        rule = rules[i]
        if rule in to_del:
            del rules[i]
            to_del.remove(rule)
        else:
            i += 1
    return rules[::-1]

def print_to_file(rules, fname_sorted):
    with open(fname_sorted, 'w', encoding = 'utf-8') as f:
        for line in rules:
            if len(line) != 0 and line[-1] == ';':
                line = line[:-1].strip()
            line = line.replace(';;', ';')
            line = line.replace('->;', '->')
            print(line, file = f)

def remove_duplicates(rules):
    res = []
    seen_rules = set()
    for rule in rules:
        if rule not in seen_rules:
            res.append(rule)
        seen_rules.add(rule)
    return res
        

processed_rules = process_rules_from_file(filename)
print_to_file(remove_duplicates(processed_rules), filename_sorted)



In [254]:
# сортировка и удаление дубликатов в файле
def printSortedNoDuplicatesFile(fname, fname_sorted):
    lines_seen = set()
    with open(fname, 'r', encoding = 'utf-8') as r:
        with open(fname_sorted, 'w', encoding = 'utf-8') as f:
            for line_orig in sorted(r):
                if line_orig not in lines_seen:
                    # чистим от пробелов
                    line = str(line_orig.strip())

                    if len(line) != 0 and line[-1] == ';':
                        # удаляю лишние ;
                        line = line[:-1].strip()
                        
                    line = line.replace(';;', ';')
                    line = line.replace('->;', '->')
                    
                    line += '\n'  
                    if line == '\n':
                        # empty line
                        continue
                    
                    print(line, end = '', file = f)
                    
                    lines_seen.add(line_orig)

In [255]:
#printSortedNoDuplicatesFile(filename, filename_sorted)

In [256]:
# печать словаря
def printDictionary(d, fname):
    with open(fname, 'w', encoding='utf-8') as f:
        for key, value in d.items():
            s = ' | '.join(value)
            print(f'[{key}] = {s}', file = f)

In [257]:
printDictionary(d, f'{corpus_filename}_dict.out')

In [258]:
d_gram_map = {}
f_gram_map = {}

In [259]:
def addToMap(mapName, key, value):
    # добавление в словарь по ключу key value (само значение - список)
    if key in mapName.keys():
        mapName[key].append(value)
    else:
        mapName[key] = [value]

def mapGrammar(filename):
    # запись всей грамматики в словарь
    # d_gram_map - словарь для D нетерминалов
    # f_gram_map - словарь для F нетерминалов
    
    with open(filename, 'r', encoding = 'utf-8') as r:
        for line in r:
            s = line.split('->')
            p = s[0].strip()
            w = s[1].strip().replace('\n', '')
            if (p[0] == 'D'):
                addToMap(d_gram_map, p, w)
            else:
                addToMap(f_gram_map, p, w)

In [260]:
def printGrammarMap(mapName, filename_min):
    # печать грамматики
    with open(filename_min, 'w', encoding = 'utf-8') as f:
        for key, value in mapName.items():
            s = '|'.join(value)
            s = ';'.join(s.split(';'))
            print(key, '->', s, file = f, sep = '')

In [261]:
root_d = []

def setFToD():
    # подстановка F нетерминалов в D
    for key, value in d_gram_map.items():
        d_gram_map[key] = f_gram_map[d_gram_map[key][0]]
        
def processRoot():
    for key, value in f_gram_map.items():
        if key[0] == 'R':
            k = key.replace('RD', 'D')
            if k in d_gram_map:
                d_gram_map[k] = value + d_gram_map[k]
            else:  
                d_gram_map[k] = value
            root_d.append(k)

def addFToD():
    for key, value in f_gram_map.items():
        newKey = key.replace('RF', 'D').replace('F', 'D')
        if newKey in d_gram_map:
            d_gram_map[newKey] = d_gram_map[newKey] + value
        else:
            d_gram_map[newKey] = value
        
        

In [262]:
def removeRecursion(d_gram_map):
    # подставить D в D где это возможно
    seen = []
    t = True
    while t: 
        t = False # закончится, если справа не останется D, или все правила просмотрены и обработаны(останутся только те, в которых есть рекурсии)
        for key, value in d_gram_map.items():
            if 'D{' not in str(value) and key not in seen: # если справа есть нетерминал D
                t = True
                seen.append(key)
                
                for k, v in d_gram_map.items():
                    l = []
                    if key in str(v):
                        for item in v:
                            for val in value:
                                l.append(item.replace(key, val)) # заменяю D на D из других правил
                    if l != []:
                        d_gram_map[k] = l
                        
    return d_gram_map
            

In [263]:
import copy

In [264]:
d_gram_map = {}
f_gram_map = {}

t_before_rsp = time.time()

mapGrammar(f'{corpus_filename}_gram_sorted.out')
#mapGrammar(f'{corpus_filename}_gram_sorted_root.out')

#setFToD()
processRoot()

printGrammarMap(d_gram_map, f'{corpus_filename}_gram_min_test.out')


#for key in f_gram_map.keys():
#    d_gram_map[key.replace('RF', 'F')] = f_gram_map[key]

new_map = d_gram_map
#new_map = removeRecursion(copy.deepcopy(d_gram_map))

printGrammarMap(new_map, f'{corpus_filename}_gram_min.out')
printSortedNoDuplicatesFile(f'{corpus_filename}_gram_min.out', f'{corpus_filename}_gram_min_sorted.out')


### Генерация рекурсивной сети переходов: <a class="anchor" id="rtn"></a>

In [265]:
descriptor = 'DESCRIPTOR'

root_rtn = []

In [266]:
def non_term_operators_no_comma(SSR = None):
    operators = [
        '@CopySSR',
        '@ToTier',
    ]
    return ' '.join(operators)

def non_term_operators_comma(SSR):
    operators = [
        f'@ToSSR("{SSR}")',
        '@CopyMainWord',
        '@CopyBonus'
    ]
    return ' '.join(operators)

def term_operators(SSR = None):
    operators = [
        '@ToMainWord',
        '' if SSR is None else f'@ToSSR("{SSR}")',
    ]
    return ' '.join(operators)

def get_ssr(term):
    return term[term.find(', ') + 2:term.find('}')]

class RTN:
    def __init__(self, name, startingNodes = [], makeStateMap = True):
        self.name = name
        self.startingNodes = startingNodes
        self.reversed = False
        
        if makeStateMap:
            self.makeStateMap()
        
    def reverse(self):
        self.reversed = not self.reversed
        new_state_map = {}
        for key, value in self.state_map.items():
            for node in value:
                ind = node.index if node.index else '*'

                if key == '*' or str(key) == '0':
                    node.index = None
                    node.isEnd = True
                else:
                    node.index = key
                    node.isEnd = False
                    
                if ind not in new_state_map:
                    new_state_map[ind] = [node]
                else:
                    new_state_map[ind].append(node)
        new_state_map[0] = new_state_map['*']
        del new_state_map['*']

        self.state_map = new_state_map  
        
        # state_map:
        """
        {
            'название_состояния': 'список RTNNode в этом состоянии',
            ...
        } 
        
        {
            '0': [
                RTNNode('a', index = 1),
                RTNNode('b', isEnd = True), 
            ],
            '1': [
                RTNNode('c', isEnd = True), 
            ]
        } 
        ===
        {'0': ['a' -> 1, 'b' -> '*'], '1': ['c' -> '*']}
        """
           
            
    # переименнование индексов и состояний
    def renameIndexes(self):
        new_state_map = {}
        index_map = {}
        new_index = 0
        
        keys = sorted(self.state_map.keys())
        keys.remove('0')
        keys = ['0'] + keys
 
        for k in keys:
            if str(k) in self.state_map:
                key = str(k)
            else:
                key = k
            
            new_key = str(new_index)
            
            
            new_state_map[new_key] = self.state_map[key]
            if '*' in key:
                new_state_map[new_key].append(RTNNode('<>', isEnd = True))
            index_map[key] = new_key
            del self.state_map[key]
            new_index += 1
        
        for value in new_state_map.values():
            for node in value:
                if node.index:
                    node.index = index_map[str(node.index)]
        check_zero_state = True
        to_state = -1
        
        for item in new_state_map['0']:
            if not item.empty:
                check_zero_state = False
                break
            if to_state == -1:
                to_state = item.index
            elif to_state != item.index:
                check_zero_state = False
                break
            
        if check_zero_state:
            new_zero = new_state_map['0'][0].index
            new_state_map['0'] = new_state_map[str(new_zero)]
            del new_state_map[str(new_zero)]
        else:
            toRemove = []
            for item in new_state_map['0']:
                if item.empty:
                    toRemove.append(item)
            if len(toRemove) != len(new_state_map['0']): 
                for item in toRemove:
                    new_state_map['0'].remove(item)
           
        
            
        self.state_map = new_state_map
        
    # в случае когда РСП строилась связыванием RTNNode друг с другом
    def makeStateMap(self):
        state_map = {}  
        
        def add_to_map(node, state_index):
            if state_index not in state_map:   
                state_map[state_index] = [node]
            else:
                state_map[state_index].append(node)
            
            for child in node.connected:
                add_to_map(child, node.index)
        
        for child in self.startingNodes:
            add_to_map(child, 0)

        self.state_map = state_map
    
    # отсоединить все RTNNode друг от друга(теперь используется только state_map)
    def disconnectAll(self):
        def disconnectNodes(node):
            for child in node.connected:
                disconnectNodes(child)
            node.disconnect()
        
        for child in self.startingNodes:
            disconnectNodes(child)
        
    def print(self):
        print(f'${self.name} {descriptor}')
        print('(')
        
        parent_ssr = None if ',' not in self.name else get_ssr(self.name)
        
        for key in self.state_map.keys():
            print(f'{key}:')
            for node in self.state_map[key]:
                node.printNode(parent_ssr)
                 
        desc_operator = '@CheckSSRAnyWord' if ',' in self.name else '@CheckTier'
        print(f') DESCRIPTOR {desc_operator}')

        print()
        
class RTNNode:
    # класс для рекурсивной сети переходов
    # s - символ перехода
    # index - в какое состояние переходим по этому нетерминалу/терминалу(цифра после)
    # isEnd - последнее ли правило в альтернативе
    # connected - с каким RTNNode связано текущее
    def __init__(self, s, isEnd = False, index = None):
        self.s = s
        self.isEnd = isEnd
        self.connected = []
        self.index = index
    
    def __str__(self):
        return str((self.s, self.connected))
    
    def __repr__(self):
        return f'{self.s} -> {self.index if self.index else "*"}'
    
    def __eq__(self, other):
        if not isinstance(other, RTNNode):
            return NotImplemented
        return (self.s, self.isEnd, self.index) == (other.s, other.isEnd, other.index)

    def __hash__(self):
        return hash((self.s, self.isEnd, self.index))
    
    @property
    def empty(self):
        return self.s == '<>'
    
    def change_index(self, ind):
        self.index = ind  
        
    def connect(self, node):
        self.connected.append(node)
        
    def disconnect(self):
        self.connected = []
        
    # печать одного RTNNode
    def printNode(self, parent_ssr = None):
        f = self.s[0] == 'D' or self.s[0] == 'F'
        s = f'${self.s}' if f else self.s
        
        state = '*' if self.isEnd else self.index
        end = (non_term_operators_no_comma() if parent_ssr == None else non_term_operators_comma(parent_ssr)) if f else term_operators() if parent_ssr == None else term_operators(parent_ssr)
        end = end.strip()
        
        if s == '<>':
            end = ''
            
        end_string =  f'{state} {end}'
        end_string = end_string.strip()
        
        print(s, end = f' {end_string}; \n')
     
    

In [267]:

# все цепочки для РСП
# не работает, если есть петли
def rtn_chain(rtn):
    def build_chain(state_map, cur_state, cur_chain):
        s = set()
        if str(cur_state) in state_map:
            state = state_map[str(cur_state)]
        elif int(cur_state) in state_map:
            state = state_map[int(cur_state)]
        else:
            state = []
            
        for node in state:
            if node.isEnd:
                if node.s == '<>':
                    s.add(cur_chain)
                else:
                    s.add(cur_chain + node.s)
            else:
                s.update(build_chain(state_map, node.index, cur_chain + node.s if node.s != '<>' else ''))
        return s
    return build_chain(rtn.state_map, 0 if 0 in rtn.state_map else '0', '')


# словарь всех переходов для каждого из состояний
# {0: {'a': [1,2], 'd': [3]}, 1: ['c': ['*']]}
# из 0 по 'a' в 1,2, по 'd' в 3 и тд 
def rtn_transition_map(rtn):
    transition_map = {}  
    
    def add_to_map(node, state_index):
        if state_index not in transition_map:   
            transition_map[state_index] = {}
        index = node.index if node.index else '*'
        
        if node.s in transition_map[state_index]:
            transition_map[state_index][node.s].append(index)
        else:
            transition_map[state_index][node.s] = [index]
    
    for key, value in rtn.state_map.items():
        for node in value:
            add_to_map(node, key)

    return transition_map

# детерминизация
# перевод РСП в словарь переходов
# детерминизация словаря переходов
# перевод детерминизированного словаря обратно в рсп
def determinize(rtn):
    transition_map = rtn_transition_map(rtn)

    determinized_transitions = determinize_nfa(transition_map)
    
    new_rtn = state_map_to_rtn(determinized_transitions, rtn.name, rtn.reversed)
    
    return new_rtn

def determinize_nfa(nfa):
    symbols = set()
    for transitions in nfa.values():
        symbols.update(transitions.keys())
    
    def dfa_transitions_string(states, nfa):
        transitions = {}
        for symbol in symbols:
            reachable = set()
            for state in states.split('_'):
                if state != '*' and state in nfa and symbol in nfa[state]:
                    reachable.update(nfa[state][symbol])
                elif state != '*' and int(state) in nfa and symbol in nfa[int(state)]:
                    reachable.update(nfa[int(state)][symbol])
            try:
                if states != '*' and states in nfa and symbol in nfa[states]:
                    reachable.update(nfa[states][symbol])
                elif states != '*' and int(states) in nfa and symbol in nfa[int(states)]:
                    reachable.update(nfa[int(states)][symbol])
            except Exception:
                pass
            
            reachable_str = '_'.join(sorted(map(str, reachable)))
            
            if reachable_str:
                transitions[symbol] = reachable_str
        return transitions

    dfa = {}

    stack = ['0', '*']
    
    while stack:
        state = stack.pop()
        transitions = dfa_transitions_string(state, nfa)

        dfa[state] = transitions

        for next_state in transitions.values():
            if next_state not in dfa:
                stack.append(next_state)
    
    dfa = {k: v for k, v in dfa.items() if v}    
    return dfa


# перевод словаря переходов обратно в рсп
def state_map_to_rtn(dfa, name, reversed):
    new_state_map = {}

    for key, value in dfa.items():
        for node in value:
            rtnnode = RTNNode(node, isEnd = dfa[key][node] == '*')

            if dfa[key][node] == '0' or dfa[key][node] == '*':
                rtnnode.index = None
                rtnnode.isEnd = True
            else:
                rtnnode.index = dfa[key][node]
                rtnnode.isEnd = False
                
            if key not in new_state_map:
                new_state_map[key] = [rtnnode]
            else:
                new_state_map[key].append(rtnnode)
                
    rtn = RTN(name)
    rtn.state_map = new_state_map

    rtn.reversed = reversed
    
    return rtn

# минимизация
# реверс -> детерминизация -> переименнование индексов и состояний -> реверс -> детерминизация -> переименнование индексов и состояний
def minimize(rtn):
    new_rtn = rtn
    new_rtn.reverse()
    new_rtn = determinize(new_rtn)
    new_rtn.renameIndexes()
    new_rtn.reverse()
    new_rtn = determinize(new_rtn)
    new_rtn.disconnectAll()
    new_rtn.renameIndexes()
    return new_rtn   

In [268]:
# визуализация РСП
# использование: visualize_state_machine(rtn.state_map)
# pdf файл открывается автоматически!
from graphviz import Digraph

def visualize_state_machine(state_map):
    dot = Digraph(comment='State Machine')

    for state, nodes in state_map.items():
        dot.node(str(state), str(state))
        for node in nodes:
            end_state = '*' if node.isEnd else node.index
            print(node.s)
            dot.edge(str(state), str(end_state), label = node.s if node.s != '<>' else 'eps')

    dot.render('state_machine.gv', view = True)

In [269]:
minimize_rtn = True # минимизировать ли РСП

count_all_states = 0 # подсчет общего числа состояний

In [270]:

def toRTN(new_map, fname):
    orig_stdout = sys.stdout
    f = open(fname, 'w', encoding = 'utf-8')
    sys.stdout = f
    
    global count_all_states
    
    for key in new_map.keys():
        l = new_map[key] # ['a;D1', 'D2;D1']
        new_l = [] # [['a', 'D1'], ['D2', 'D1']]
        for item in l:
            p = item.split(';')
            new_l.append(p)
            
        
        
        start_ones = []
        
        for items in new_l:
            
            last = None # RTNNode на предыдущей итерации
            
            
            for i, item in enumerate(reversed(items)):
                # цикл по всем терминалам и нетерминалам в альтернативе в обратном порядке
                r = RTNNode(item, isEnd = i == 0)
                
                if last != None:
                    r.connect(last) # связываем текущий с предыдущей итерацией
                    
                last = r
                
            start_ones.append(last)
        
        l = 1
        for r_node in start_ones:
            r_tmp = r_node
            while not r_tmp.isEnd:
                r_tmp.change_index(l)
                l += 1
                r_tmp = r_tmp.connected[0]
        
        rtn_network = RTN(key, start_ones)
        
        
        if key in root_d:
            root_rtn.append(rtn_network)
            
        if minimize_rtn:
            chain1 = rtn_chain(rtn_network)
            
            rtn_network = minimize(rtn_network)
            
            chain2 = rtn_chain(rtn_network)
            #if len(chain2.symmetric_difference(chain1)) != 0:
                # если цепочки не совпадает, что-то пошло не так при минимизации
            #    print('!!!')
            
        rtn_network.print()
        
        count_all_states += len(rtn_network.state_map)
        
    sys.stdout = orig_stdout
    f.close()

In [271]:
def root_descriptor():
    return 'DECLARE DESCRIPTOR (MAINWORD TIER() SSR{""} BONUS{0})\n\n'

def root_operators():
    operators = [
        '@CopySSR',
        '@CopyMainWord',
        '@CopyBonus'
    ]
    return ' '.join(operators)

def add_root_to_file(root_d, filename):
    with open(filename, 'r', encoding='utf8') as contents:
      save = contents.read()
    with open(filename, 'w', encoding='utf8') as contents:
        contents.write(root_descriptor())
        contents.write('$D{' + orig_name + '}\n')
        contents.write('(\n')
        contents.write('0:\n')
        for root in root_d:
            contents.write('$' + root + f' * {root_operators()};\n')
        contents.write(') DESCRIPTOR\n\n')
    with open(filename, 'a', encoding='utf8') as contents:
        contents.write(save)

In [272]:
toRTN(new_map, f'{corpus_filename}_rtn.out')

In [273]:
f'Количество состояний в рекурсивной сети переходов = {len(new_map)}'

'Количество состояний в рекурсивной сети переходов = 85'

In [274]:
f'Общее количество состояний в рекурсивной сети переходов = {count_all_states}'

'Общее количество состояний в рекурсивной сети переходов = 247'

In [275]:
add_root_to_file(root_d, f'{corpus_filename}_rtn.out')

In [276]:
f'Время, потраченное на генерацию РСП - {time.time() - t_before_rsp}'

'Время, потраченное на генерацию РСП - 0.1659984588623047'

In [277]:
# разные тесты на минимизацию

test_minimize = False
test_minimize_2 = False
test_minimize_3 = False
test_minimize_4 = True

In [286]:
if test_minimize:
    name = "exampleNFA"
    s1 = RTNNode("a")
    s2 = RTNNode("a")
    s3 = RTNNode("b", isEnd = True)
    s4 = RTNNode("c", isEnd = True)
    s7 = RTNNode("f", isEnd = True)
    s5 = RTNNode("d")
    s6 = RTNNode("e", isEnd = True)

    s1.index = 1
    s2.index = 2
    s5.index = 3
    s1.connect(s4)
    s1.connect(s7)
    s2.connect(s3)
    s5.connect(s6)

    rtn = RTN(name, [s1, s2, s5])
    chain1 = rtn_chain(rtn)
    print(chain1)
    
    new_rtn = minimize(rtn)
    print(new_rtn.state_map)
    chain = rtn_chain(new_rtn)
    print(chain)
    
    print(chain == rtn_chain(rtn))
    
    print(len(chain.symmetric_difference(chain1)))

if test_minimize_2:
    name = "$D{[PART]}"
    s1 = RTNNode("a")
    s2 = RTNNode("b")
    s3 = RTNNode("c")
    s4 = RTNNode("c", isEnd=True)
    s5 = RTNNode("c", isEnd=True)
    s6 = RTNNode("b", isEnd=True)
    
    s1.index = 1
    s2.index = 2
    s3.index = 3
    
    s1.connect(s4)
    s2.connect(s5)
    s3.connect(s6)
    
    
    
    rtn = RTN(name, [s1, s2, s3])
    chain1 = rtn_chain(rtn)
    print(chain1)
    
    new_rtn = minimize(rtn)
    print(new_rtn.state_map)
    chain = rtn_chain(new_rtn)
    print(chain)
    
    print(chain == rtn_chain(rtn))
    
    print(len(chain.symmetric_difference(chain1)))

if test_minimize_3:
    name = "$D"
    s1 = RTNNode("a")
    s1.index = 1
    s2 = RTNNode("b")
    s2.index = 3
    s3 = RTNNode("b")
    s3.index = 5
    s4 = RTNNode("c")
    s4.index = 8
    s5 = RTNNode("c")
    s5.index = 9
    s6 = RTNNode("c")
    s6.index = 10
    s7 = RTNNode("c")
    s7.index = 12
    s8 = RTNNode("d")
    s8.index = 14
    s9 = RTNNode("c")
    s9.index = 16
    s10 = RTNNode("c")
    s10.index = 18
    
    s11 = RTNNode("c")
    s11.index = 2
    
    s12 = RTNNode("e")
    s12.isEnd = True
    
    s13 = RTNNode("c")
    s13.index = 4
    
    s14 = RTNNode("e")
    s14.isEnd = True
    
    s15 = RTNNode("c")
    s15.index = 6
    
    s16 = RTNNode("e")
    s16.index = 7
    
    s17 = RTNNode("a")
    s17.isEnd = True
    
    s18 = RTNNode("e")
    s18.isEnd = True
    
    s19 = RTNNode("f")
    s19.isEnd = True
    
    s20 = RTNNode("e")
    s20.index = 11
    
    s21 = RTNNode("a")
    s21.isEnd = True
    
    s22 = RTNNode("f")
    s22.index = 13
    
    s23 = RTNNode("g")
    s23.isEnd = True
    
    s24 = RTNNode("c")
    s24.index = 15
    
    s25 = RTNNode("e")
    s25.isEnd = True
    
    s26 = RTNNode("e")
    s26.index = 17
    
    s27 = RTNNode("h")
    s27.isEnd = True
    
    s28 = RTNNode("i")
    s28.isEnd = True
    
    state_m = {
        '0': [
            s1, s2, s3, s4, s5, s6, s7, s8, s9, s10
        ],
        '1': [s11],
        '2': [s12],
        '3': [s13],
        '4': [s14],
        '5': [s15],
        '6': [s16],
        '7': [s17],
        '8': [s18],
        '9': [s19],
        '10': [s20],
        '11': [s21],
        '12': [s22],
        '13': [s23],
        '14': [s24],
        '15': [s25],
        '16': [s26],
        '17': [s27], 
        '18': [s28],
    }
    
    rtn = RTN(name)
    rtn.state_map = state_m
    #chain1 = rtn_chain(rtn)
    #print(rtn.state_map)
    #print(chain1)
    #visualize_state_machine(rtn.state_map)
    
    new_rtn = minimize(rtn)
    print(new_rtn.state_map)
    visualize_state_machine(new_rtn.state_map)
    #chain = rtn_chain(new_rtn)
    #print(chain)
    
    #print(chain == rtn_chain(rtn))
    
    #print(chain.symmetric_difference(chain1))
    #print(len(chain.symmetric_difference(chain1)))
    
    
if test_minimize_4:
    name = 'test'
    # 0 - H
    # 1 - A
    # 2 - B
    # 3 - C
    # 4 - D
    # 5 - E
    # 6 - F
    
    st_map = {
        '0': [
            RTNNode('a', index = 1),
            RTNNode('a', index = 2), 
        ],
        '1': [
            RTNNode('a', index = 3),
        ],
        '2': [
            RTNNode('a', index = 2),
            RTNNode('b', index = 4),
            RTNNode('b', isEnd = True),
        ],
        '3': [
            RTNNode('a', index = 2),
            RTNNode('a', index = 5),
        ],
        '4': [
            RTNNode('b', index = 4),
            RTNNode('b', index = 5),
            RTNNode('b', isEnd = True),
        ],
        '5': [
            RTNNode('b', index = 6),
        ],
        '6': [
            RTNNode('b', isEnd = True),
        ],
    }
    
    rtn = RTN(name)
    rtn.state_map = st_map
    print(rtn.state_map)
    #visualize_state_machine(rtn.state_map)
    new_rtn = minimize(rtn)
    print(new_rtn.state_map)
    
    #rtn = RTN(name)
    #rtn.state_map = {
    #    '0': [
    #        RTNNode('<>', index = 1),
    #        RTNNode('<>', index = 1),
    #        RTNNode('<>', index = 2)
    #    ],
    #    '1': [
    #        RTNNode('a', isEnd = True)
    #    ],
    #    '2': [
    #        RTNNode('b', isEnd = True)
    #    ]
    #}
    #rtn.renameIndexes()
    #print(rtn.state_map)
    
    visualize_state_machine(new_rtn.state_map)


{'0': [a -> 1, a -> 2], '1': [a -> 3], '2': [a -> 2, b -> 4, b -> *], '3': [a -> 2, a -> 5], '4': [b -> 4, b -> 5, b -> *], '5': [b -> 6], '6': [b -> *]}
{'0': [a -> 3], '1': [b -> 1, <> -> *], '3': [b -> 1, a -> 3]}
a
b
<>
b
a


### Генерация JSON <a class="anchor" id="json"></a>

In [279]:
import json
import pymorphy2
from tqdm import tqdm

morph = pymorphy2.MorphAnalyzer()

with open('pym_to_synt/pymorphy_to_syntagrus.json', 'r', encoding='utf-8') as f:
    pts_m = json.load(f)
    

In [280]:
def to_syntagrus(word):
    m = morph.parse(word)

    l = []
    for p in m:
        t = set(p.tag.grammemes)
        l.append(t)
        
    res = []
    for item in l:
        res_item = []
        for tag in item:
            if tag in pts_m:
                res_item += pts_m[tag]
        res.append(list(set(res_item)))
    return res

In [281]:
def json_map():

    l = []
    exclude = ['МЕТА', 'НАСТ', 'СЛ', 'COM']
    for s in sentences:
        sent = Sentence(s)
        if sent.id in non_projective:
            continue
        words = sent.words
        
        w_l = []
        s_l = []
        for w in words:
            grammems = [w.feat.split(' ')]
            for item in exclude:
                if item in grammems[0]:
                    grammems[0].remove(item)
            if len(grammems[0]) == 0:
                grammems[0].append('S')
            v = w.v
            
            s_l.append(v)
            
            w_m = {
                "word": v,
                "FEAT": grammems
            }
            w_l.append(w_m)
        s_m = {
            "sentence": ' '.join(s_l),
            "words": w_l
        }
        l.append(s_m)
    return l

In [282]:
l = json_map()

with open(f'{corpus_filename}_input.json', 'w', encoding='utf-8') as f:
    json.dump(l, f, ensure_ascii = False, indent = 4)

In [283]:
f'Количество ключей в json разметке - {len(l)}'

'Количество ключей в json разметке - 74'

* [В начало](#contents)